# Galaxy plots

This version matches galaxies with both subhalos and halos.  
However at this stage, there are no properties stored in either subhalos or halos apart from mass (I think).

In [ ]:
# Imports
import astropy.constants as c
import astropy.units as u
import h5py
h5py.enable_ipython_completer()
import numpy as np
import yaml
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_context('poster')
sns.set_style('whitegrid')

In [ ]:
# Script parameters
file_parameters='galaxy.yml'
displayParameters=True

# Read parameters from yaml file
parameters=yaml.load(open(file_parameters),Loader=yaml.FullLoader)
if displayParameters:
    for item in parameters:
        print("{:20s}: {}".format(item,parameters[item]))

haloFile=parameters['inputFiles']['haloFile']
subhaloFile=parameters['inputFiles']['subhaloFile']
galaxyFile=parameters['inputFiles']['galaxyFile']
graphMin=parameters['graphs']['graphMin']
graphMax=parameters['graphs']['graphMax']
snapMin=parameters['snapshots']['snapMin']
snapMax=parameters['snapshots']['snapMax']
baryonFraction=parameters['cosmo']['baryonFraction']

In [ ]:
# Read in halo table
with h5py.File(haloFile,'r') as f:
    # Get the data from the snapshot
    halos=f['Halos'][:]
    print('halos.dtype =',halos.dtype)
    
# Read in subhalo table
with h5py.File(subhaloFile,'r') as f:
    # Get the data from the snapshot
    subs=f['Halos'][:]
    print('subs.dtype =',subs.dtype)

# Read in galaxy table
with h5py.File(galaxyFile,'r') as f:    
    # Get the data from the snapshot
    gals=f['Galaxies'][:]
    print('gals.dtype =',gals.dtype)
    # Get the starting locations of each graph in the galaxy table
    graph_start=f['Graph_start_locations'][:]


In [ ]:
# Filter out the galaxies in our selection.
# Might be faster to include np.nonzero() on rhs, but this packages indices in a tuple
gals=gals[(graphMin<=gals['graph_ID']) & (gals['graph_ID']<=graphMax) & 
            (snapMin<=gals['snap_ID']) & (gals['snap_ID']<=snapMax) & gals['b_exists']==True]
n_gal=len(gals)

In [ ]:
# Match up galaxies with subhalos and/or halos
# Note that orphan galaxies can have no halo if there is no descendant for the host halo in the previous snap.
# Note that orphan galaxies have no subhalo.
b_first_warning=True
halo_loc=np.zeros(n_gal,dtype=np.int32)
sub_loc=np.zeros(n_gal,dtype=np.int32)
mask_halo_exists=np.full(n_gal,True,dtype=bool)
mask_sub_exists=np.full(n_gal,True,dtype=bool)
for i_gal in range(n_gal):
    matches=np.where(np.logical_and(halos['graph_ID']==gals['graph_ID'][i_gal],halos['halo_gid']==gals['halo_gid'][i_gal]))[0]
    if len(matches)!=1:
        mask_halo_exists[i_gal]=False
    else:
        halo_loc[i_gal]=matches[0]
    matches=np.where(np.logical_and(subs['graph_ID']==gals['graph_ID'][i_gal],subs['sub_gid']==gals['sub_gid'][i_gal]))[0]   
    if len(matches)!=1:
        mask_sub_exists[i_gal]=False
    else:
        sub_loc[i_gal]=matches[0]
halos=halos[halo_loc]
subs=subs[sub_loc]

In [ ]:
for plot in parameters['plots'].items():
    print(plot)
    if plot[1]:
        plotName=plot[0]
        plotProps=parameters[plotName]
        print(plotProps)
        plt.figure(figsize=plotProps['figsize'])
        plt.xscale(plotProps['xscale'])
        plt.yscale(plotProps['yscale'])
        plt.title(plotName)
        plt.xlabel(plotProps['xlabel'])
        plt.ylabel(plotProps['ylabel'])
        if plotName=='coldGas_halo':
            xdata=halos['mass'][mask_halo_exists]
            ydata=gals['mass_gas_cold'][mask_halo_exists]
            plt.plot(xdata,xdata*baryonFraction,'-',label='cosmic mean baryon fraction')
        elif plotName=='coldGas_subhalo':
            xdata=subs['mass'][mask_sub_exists]
            ydata=gals['mass_gas_cold'][mask_sub_exists]
            plt.plot(xdata,xdata*baryonFraction,'-',label='cosmic mean baryon fraction')
        elif plotName=='stellarMass':
            xdata=subhalos['mass']
            ydata=np.zeros(len(subhalos))
            for i_sub in range(len(subhalos)):
                ydata[i_sub]=np.sum(galaxies['mass_stars'][np.where(galaxies['sub_gid']==subhalos['sub_gid'][i_sub])])
            plt.grid(which='minor',visible=True)
        elif plotName=='stellarHaloMassRatio':
            xdata=subhalos['mass']
            ydata=np.zeros(len(subhalos))
            for i_sub in range(len(subhalos)):
                ydata[i_sub]=np.sum(galaxies['mass_stars'][np.where(galaxies['sub_gid']==subhalos['sub_gid'][i_sub])])
            ydata=ydata/subhalos['mass']
#         elif plotName=='starFormationRate':
#             xdata=halos['mass']
#             ydata=halos['star_formation_rate']
        else:
            print('Plot type',plotName,'not yet implemented')
            continue # with next plot request
        plt.plot(xdata,ydata,plotProps['markerType'],label=plotProps['label'])
        if plotProps['legend']: plt.legend()
        if plotProps['save_png']: plt.savefig('figs/'+plotProps['name']+'.png',bbox_inches='tight')
        if plotProps['save_pdf']: plt.savefig('figs/'+plotProps['name']+'.pdf',bbox_inches='tight')

In [ ]:
i_gal=np.where(np.logical_and(ydata>1e-10,ydata<10))[0]
print(gals.dtype)
print((gals[mask_halo_exists])[i_gal])

In [ ]:
len(gals)